# PitWall Live - Pit Stop Strategy Optimization

This notebook develops models for optimal pit stop strategy:
1. Optimal pit window prediction
2. Undercut/overcut opportunity detection
3. Tire compound selection optimization
4. Multi-stop strategy analysis
5. Safety car strategy adaptation

## Model Objectives
- Predict optimal pit stop windows based on tire degradation
- Identify undercut/overcut opportunities in real-time
- Recommend tire compound for each stint
- Evaluate 1-stop vs 2-stop vs 3-stop strategies

In [ ]:
import os
import sys
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from tqdm.notebook import tqdm
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass
from enum import Enum

import fastf1
from fastf1 import get_session, get_event_schedule

from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, mean_absolute_error

import lightgbm as lgb
import xgboost as xgb

# Configure plotting
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 11

# Enable FastF1 caching
CACHE_DIR = Path('../data/cache')
CACHE_DIR.mkdir(parents=True, exist_ok=True)
fastf1.Cache.enable_cache(str(CACHE_DIR))

print("Pit Stop Strategy Model - Setup Complete")

## 1. Data Structures and Constants

In [ ]:
class TireCompound(Enum):
    """Tire compound enumeration."""
    SOFT = 'SOFT'
    MEDIUM = 'MEDIUM'
    HARD = 'HARD'
    INTERMEDIATE = 'INTERMEDIATE'
    WET = 'WET'


@dataclass
class PitStopEvent:
    """Represents a pit stop event."""
    driver: str
    lap: int
    pit_duration: float  # seconds
    tire_in: str
    tire_out: str
    tire_age_in: int
    position_before: int
    position_after: int
    time_lost: float  # seconds lost compared to staying out


@dataclass
class RaceStrategy:
    """Complete race strategy for a driver."""
    driver: str
    pit_stops: List[PitStopEvent]
    stints: List[Dict]
    total_pit_time: float
    final_position: int
    positions_gained: int


# Typical pit lane time loss (varies by circuit)
CIRCUIT_PIT_LOSS = {
    'Bahrain': 21.0,
    'Jeddah': 22.5,
    'Melbourne': 19.5,
    'Suzuka': 24.0,
    'Shanghai': 21.0,
    'Miami': 23.0,
    'Monaco': 23.5,
    'Montreal': 20.5,
    'Barcelona': 21.0,
    'Spielberg': 19.0,
    'Silverstone': 20.0,
    'Budapest': 21.5,
    'Spa': 22.0,
    'Zandvoort': 18.5,
    'Monza': 24.0,
    'Singapore': 28.0,
    'Austin': 21.5,
    'Mexico City': 20.0,
    'Interlagos': 20.5,
    'Las Vegas': 23.0,
    'Lusail': 22.0,
    'Yas Marina': 21.0
}

# Average tire life (laps) by compound
TIRE_LIFE_ESTIMATE = {
    'SOFT': {'min': 12, 'optimal': 18, 'max': 25},
    'MEDIUM': {'min': 20, 'optimal': 28, 'max': 38},
    'HARD': {'min': 30, 'optimal': 40, 'max': 50}
}

print("Data structures defined")

## 2. Load and Process Pit Stop Data

In [ ]:
class PitStopDataLoader:
    """Load and process pit stop data from races."""
    
    def __init__(self, cache_dir: Path = CACHE_DIR):
        self.cache_dir = cache_dir
    
    def load_race_pit_stops(self, year: int, event_name: str) -> pd.DataFrame:
        """Load pit stop data for a race."""
        try:
            session = get_session(year, event_name, 'R')
            session.load()
            
            laps = session.laps
            results = session.results
            
            # Identify pit stops from lap data
            pit_stops = []
            
            for driver in laps['Driver'].unique():
                driver_laps = laps[laps['Driver'] == driver].sort_values('LapNumber')
                
                # Find pit in laps
                pit_in_laps = driver_laps[driver_laps['PitInTime'].notna()]
                
                for _, pit_lap in pit_in_laps.iterrows():
                    lap_num = pit_lap['LapNumber']
                    
                    # Get next lap for pit out info
                    next_lap = driver_laps[driver_laps['LapNumber'] == lap_num + 1]
                    
                    if len(next_lap) > 0:
                        next_lap = next_lap.iloc[0]
                        
                        # Calculate pit duration
                        if pd.notna(pit_lap['PitInTime']) and pd.notna(next_lap['PitOutTime']):
                            pit_duration = (next_lap['PitOutTime'] - pit_lap['PitInTime']).total_seconds()
                        else:
                            pit_duration = np.nan
                        
                        pit_stop = {
                            'Driver': driver,
                            'Lap': lap_num,
                            'PitDuration': pit_duration,
                            'TireIn': pit_lap.get('Compound', 'UNKNOWN'),
                            'TireOut': next_lap.get('Compound', 'UNKNOWN'),
                            'TireAgeIn': pit_lap.get('TyreLife', np.nan),
                            'Season': year,
                            'Event': event_name
                        }
                        pit_stops.append(pit_stop)
            
            return pd.DataFrame(pit_stops)
            
        except Exception as e:
            print(f"Error loading pit stops for {event_name} {year}: {e}")
            return pd.DataFrame()
    
    def load_race_stints(self, year: int, event_name: str) -> pd.DataFrame:
        """Load stint information for all drivers."""
        try:
            session = get_session(year, event_name, 'R')
            session.load()
            
            laps = session.laps
            stints_data = []
            
            for driver in laps['Driver'].unique():
                driver_laps = laps[laps['Driver'] == driver].sort_values('LapNumber')
                
                if 'Compound' not in driver_laps.columns:
                    continue
                
                # Identify stint boundaries
                driver_laps = driver_laps.copy()
                driver_laps['StintNumber'] = (
                    (driver_laps['Compound'] != driver_laps['Compound'].shift()) |
                    (driver_laps['TyreLife'] < driver_laps['TyreLife'].shift())
                ).cumsum()
                
                for stint_num in driver_laps['StintNumber'].unique():
                    stint_laps = driver_laps[driver_laps['StintNumber'] == stint_num]
                    
                    stint_laps['LapTimeSeconds'] = stint_laps['LapTime'].dt.total_seconds()
                    
                    # Filter out pit laps for lap time analysis
                    clean_laps = stint_laps[
                        (stint_laps['PitInTime'].isna()) &
                        (stint_laps['PitOutTime'].isna()) &
                        (stint_laps['LapTimeSeconds'] > 60) &
                        (stint_laps['LapTimeSeconds'] < 180)
                    ]
                    
                    if len(clean_laps) > 0:
                        stint = {
                            'Driver': driver,
                            'StintNumber': stint_num,
                            'Compound': stint_laps['Compound'].iloc[0],
                            'StartLap': stint_laps['LapNumber'].min(),
                            'EndLap': stint_laps['LapNumber'].max(),
                            'StintLength': len(stint_laps),
                            'AvgLapTime': clean_laps['LapTimeSeconds'].mean(),
                            'BestLapTime': clean_laps['LapTimeSeconds'].min(),
                            'LapTimeStd': clean_laps['LapTimeSeconds'].std(),
                            'Season': year,
                            'Event': event_name
                        }
                        
                        # Calculate degradation if enough laps
                        if len(clean_laps) >= 5:
                            # Linear fit for degradation
                            x = clean_laps['TyreLife'].values
                            y = clean_laps['LapTimeSeconds'].values
                            
                            if len(np.unique(x)) > 1:
                                slope, _ = np.polyfit(x, y, 1)
                                stint['DegradationRate'] = slope
                            else:
                                stint['DegradationRate'] = np.nan
                        else:
                            stint['DegradationRate'] = np.nan
                        
                        stints_data.append(stint)
            
            return pd.DataFrame(stints_data)
            
        except Exception as e:
            print(f"Error loading stints for {event_name} {year}: {e}")
            return pd.DataFrame()


# Initialize loader
pit_loader = PitStopDataLoader()

In [ ]:
# Load sample race data
sample_pit_stops = pit_loader.load_race_pit_stops(2024, 'Bahrain')
print("Pit Stops:")
sample_pit_stops.head(15)

In [ ]:
# Load stint data
sample_stints = pit_loader.load_race_stints(2024, 'Bahrain')
print("Stints:")
sample_stints.head(15)

## 3. Build Training Dataset

In [ ]:
def load_strategy_dataset(years: List[int], max_races: int = None) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Load pit stop and stint data for multiple seasons."""
    all_pit_stops = []
    all_stints = []
    
    for year in years:
        schedule = get_event_schedule(year)
        race_events = schedule[schedule['EventFormat'] != 'testing']
        
        if max_races:
            race_events = race_events.head(max_races)
        
        for _, event in tqdm(race_events.iterrows(), 
                             total=len(race_events), 
                             desc=f"Loading {year}"):
            pit_stops = pit_loader.load_race_pit_stops(year, event['EventName'])
            stints = pit_loader.load_race_stints(year, event['EventName'])
            
            if len(pit_stops) > 0:
                pit_stops['Circuit'] = event['Location']
                pit_stops['TotalLaps'] = event.get('TotalLaps', 57)  # Default estimate
                all_pit_stops.append(pit_stops)
            
            if len(stints) > 0:
                stints['Circuit'] = event['Location']
                all_stints.append(stints)
    
    pit_stops_df = pd.concat(all_pit_stops, ignore_index=True) if all_pit_stops else pd.DataFrame()
    stints_df = pd.concat(all_stints, ignore_index=True) if all_stints else pd.DataFrame()
    
    return pit_stops_df, stints_df


# Load training data (limited for demo)
print("Loading strategy dataset...")
pit_stops_df, stints_df = load_strategy_dataset([2023], max_races=5)
print(f"\nLoaded {len(pit_stops_df)} pit stops and {len(stints_df)} stints")

## 4. Pit Window Prediction Model

In [ ]:
class PitWindowPredictor:
    """Predict optimal pit stop windows based on tire degradation and race state."""
    
    def __init__(self):
        self.model = None
        self.scaler = StandardScaler()
        self.compound_encoder = LabelEncoder()
        
    def prepare_features(self, stints_df: pd.DataFrame) -> pd.DataFrame:
        """Prepare features for pit window prediction."""
        features = stints_df.copy()
        
        # Encode compound
        features['CompoundEncoded'] = self.compound_encoder.fit_transform(
            features['Compound'].fillna('UNKNOWN')
        )
        
        # Add derived features
        features['NormalizedStintLength'] = features['StintLength'] / features.groupby('Event')['StintLength'].transform('max')
        
        # Degradation indicator
        features['HighDegradation'] = (features['DegradationRate'] > 0.05).astype(int)
        
        return features
    
    def fit(self, stints_df: pd.DataFrame):
        """Train the pit window prediction model."""
        features = self.prepare_features(stints_df)
        
        # Target: optimal stint length (EndLap as proxy)
        feature_cols = ['CompoundEncoded', 'DegradationRate', 'AvgLapTime', 'StintNumber']
        
        # Remove rows with missing values
        clean_features = features.dropna(subset=feature_cols + ['StintLength'])
        
        if len(clean_features) < 10:
            print("Insufficient data for training")
            return
        
        X = clean_features[feature_cols]
        y = clean_features['StintLength']
        
        # Train LightGBM regressor
        self.model = lgb.LGBMRegressor(
            n_estimators=100,
            max_depth=6,
            learning_rate=0.1,
            random_state=42
        )
        
        self.model.fit(X, y)
        print(f"Pit window model trained on {len(X)} samples")
        
    def predict_optimal_stint_length(self, 
                                     compound: str, 
                                     degradation_rate: float,
                                     avg_lap_time: float,
                                     stint_number: int) -> Dict:
        """Predict optimal stint length for given conditions."""
        if self.model is None:
            return self._rule_based_prediction(compound)
        
        compound_encoded = self.compound_encoder.transform([compound])[0]
        
        X = np.array([[compound_encoded, degradation_rate, avg_lap_time, stint_number]])
        
        predicted_length = self.model.predict(X)[0]
        
        # Apply bounds based on tire life estimates
        tire_limits = TIRE_LIFE_ESTIMATE.get(compound, TIRE_LIFE_ESTIMATE['MEDIUM'])
        
        optimal_length = np.clip(predicted_length, tire_limits['min'], tire_limits['max'])
        
        return {
            'predicted_stint_length': round(optimal_length),
            'pit_window_start': round(optimal_length * 0.8),
            'pit_window_end': round(optimal_length * 1.1),
            'confidence': 'high' if abs(predicted_length - optimal_length) < 3 else 'medium'
        }
    
    def _rule_based_prediction(self, compound: str) -> Dict:
        """Fallback rule-based prediction."""
        tire_limits = TIRE_LIFE_ESTIMATE.get(compound, TIRE_LIFE_ESTIMATE['MEDIUM'])
        optimal = tire_limits['optimal']
        
        return {
            'predicted_stint_length': optimal,
            'pit_window_start': tire_limits['min'],
            'pit_window_end': tire_limits['max'],
            'confidence': 'low'
        }


# Train pit window predictor
pit_window_model = PitWindowPredictor()
if len(stints_df) > 0:
    pit_window_model.fit(stints_df)

In [ ]:
# Test pit window predictions
test_scenarios = [
    {'compound': 'SOFT', 'degradation_rate': 0.08, 'avg_lap_time': 92.5, 'stint_number': 1},
    {'compound': 'MEDIUM', 'degradation_rate': 0.04, 'avg_lap_time': 93.0, 'stint_number': 2},
    {'compound': 'HARD', 'degradation_rate': 0.02, 'avg_lap_time': 94.0, 'stint_number': 1},
]

print("Pit Window Predictions:")
print("=" * 60)

for scenario in test_scenarios:
    prediction = pit_window_model.predict_optimal_stint_length(**scenario)
    print(f"\n{scenario['compound']} (Deg: {scenario['degradation_rate']:.2f}s/lap):")
    print(f"  Optimal stint: {prediction['predicted_stint_length']} laps")
    print(f"  Pit window: Laps {prediction['pit_window_start']} - {prediction['pit_window_end']}")
    print(f"  Confidence: {prediction['confidence']}")

## 5. Undercut/Overcut Opportunity Detection

In [ ]:
class UndercutAnalyzer:
    """Analyze and predict undercut/overcut opportunities."""
    
    def __init__(self):
        self.undercut_model = None
        self.feature_cols = []
        
    def calculate_undercut_effectiveness(self,
                                         session,
                                         driver: str,
                                         target_driver: str,
                                         pit_lap: int) -> Dict:
        """Calculate potential effectiveness of an undercut attempt."""
        laps = session.laps
        
        driver_laps = laps[laps['Driver'] == driver].copy()
        target_laps = laps[laps['Driver'] == target_driver].copy()
        
        if len(driver_laps) == 0 or len(target_laps) == 0:
            return {'success': False, 'reason': 'Missing lap data'}
        
        # Get current gap
        current_lap = driver_laps[driver_laps['LapNumber'] == pit_lap]
        target_current = target_laps[target_laps['LapNumber'] == pit_lap]
        
        if len(current_lap) == 0 or len(target_current) == 0:
            return {'success': False, 'reason': 'Invalid lap number'}
        
        # Estimate positions
        driver_pos = current_lap['Position'].iloc[0] if 'Position' in current_lap.columns else np.nan
        target_pos = target_current['Position'].iloc[0] if 'Position' in target_current.columns else np.nan
        
        # Typical undercut gain on fresh tires
        tire_in = current_lap.get('Compound', pd.Series(['UNKNOWN'])).iloc[0]
        tire_age = current_lap.get('TyreLife', pd.Series([15])).iloc[0]
        
        # Estimate undercut gain based on tire age
        undercut_gain = self._estimate_undercut_gain(tire_age, tire_in)
        
        return {
            'success': True,
            'driver_position': driver_pos,
            'target_position': target_pos,
            'estimated_time_gain': undercut_gain,
            'tire_age': tire_age,
            'recommendation': 'UNDERCUT' if undercut_gain > 1.5 else 'HOLD'
        }
    
    def _estimate_undercut_gain(self, tire_age: int, compound: str) -> float:
        """Estimate time gain from undercut based on tire age."""
        # Base gain from fresh tire advantage
        base_gain = 0.8  # seconds per lap
        
        # Additional gain from older tires
        age_factor = min(tire_age / 10, 2.0)  # Max 2x multiplier
        
        # Compound factor
        compound_factors = {'SOFT': 1.3, 'MEDIUM': 1.0, 'HARD': 0.7}
        compound_mult = compound_factors.get(compound, 1.0)
        
        return base_gain * age_factor * compound_mult
    
    def analyze_race_undercuts(self, session) -> pd.DataFrame:
        """Analyze all undercut opportunities in a race."""
        laps = session.laps
        results = session.results
        
        undercut_events = []
        
        # Find all pit stops
        for driver in laps['Driver'].unique():
            driver_laps = laps[laps['Driver'] == driver].sort_values('LapNumber')
            pit_in_laps = driver_laps[driver_laps['PitInTime'].notna()]
            
            for _, pit_lap in pit_in_laps.iterrows():
                lap_num = pit_lap['LapNumber']
                
                # Check positions before and after pit
                pre_pit = driver_laps[driver_laps['LapNumber'] == lap_num - 1]
                post_pit = driver_laps[driver_laps['LapNumber'] == lap_num + 2]  # After out lap
                
                if len(pre_pit) > 0 and len(post_pit) > 0:
                    if 'Position' in pre_pit.columns and 'Position' in post_pit.columns:
                        pos_before = pre_pit['Position'].iloc[0]
                        pos_after = post_pit['Position'].iloc[0]
                        
                        undercut_events.append({
                            'Driver': driver,
                            'PitLap': lap_num,
                            'PositionBefore': pos_before,
                            'PositionAfter': pos_after,
                            'PositionsGained': pos_before - pos_after,
                            'TireAge': pit_lap.get('TyreLife', np.nan),
                            'Compound': pit_lap.get('Compound', 'UNKNOWN')
                        })
        
        return pd.DataFrame(undercut_events)


# Initialize analyzer
undercut_analyzer = UndercutAnalyzer()
print("Undercut analyzer initialized")

In [ ]:
# Analyze undercuts from sample race
session = get_session(2024, 'Bahrain', 'R')
session.load()

undercut_results = undercut_analyzer.analyze_race_undercuts(session)
print("Undercut Analysis - Bahrain 2024:")
undercut_results.sort_values('PositionsGained', ascending=False).head(10)

In [ ]:
# Visualize undercut effectiveness
if len(undercut_results) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Positions gained distribution
    axes[0].hist(undercut_results['PositionsGained'], bins=20, edgecolor='black')
    axes[0].axvline(x=0, color='red', linestyle='--', label='No change')
    axes[0].set_xlabel('Positions Gained/Lost')
    axes[0].set_ylabel('Count')
    axes[0].set_title('Pit Stop Position Changes')
    axes[0].legend()
    
    # Tire age vs positions gained
    scatter = axes[1].scatter(undercut_results['TireAge'], 
                              undercut_results['PositionsGained'],
                              c=undercut_results['Compound'].map({'SOFT': 'red', 'MEDIUM': 'yellow', 'HARD': 'white'}),
                              edgecolors='black', s=50, alpha=0.7)
    axes[1].axhline(y=0, color='gray', linestyle='--')
    axes[1].set_xlabel('Tire Age at Pit Stop')
    axes[1].set_ylabel('Positions Gained')
    axes[1].set_title('Tire Age vs Pit Stop Effectiveness')
    
    plt.tight_layout()
    plt.show()

## 6. Strategy Simulation Engine

In [ ]:
class StrategySimulator:
    """Simulate different pit stop strategies."""
    
    def __init__(self, total_laps: int, pit_loss: float = 21.0):
        self.total_laps = total_laps
        self.pit_loss = pit_loss
        
    def calculate_stint_time(self,
                             compound: str,
                             start_lap: int,
                             stint_length: int,
                             base_lap_time: float,
                             fuel_effect: float = 0.03) -> float:
        """Calculate total time for a stint including degradation."""
        # Degradation rates per compound (seconds per lap)
        deg_rates = {'SOFT': 0.08, 'MEDIUM': 0.05, 'HARD': 0.03}
        deg_rate = deg_rates.get(compound, 0.05)
        
        total_time = 0
        for lap in range(stint_length):
            actual_lap = start_lap + lap
            
            # Tire degradation
            tire_deg = deg_rate * lap
            
            # Fuel effect (lighter = faster)
            fuel_benefit = fuel_effect * actual_lap
            
            lap_time = base_lap_time + tire_deg - fuel_benefit
            total_time += lap_time
        
        return total_time
    
    def simulate_strategy(self,
                          stints: List[Dict],
                          base_lap_time: float) -> Dict:
        """Simulate a complete race strategy.
        
        Args:
            stints: List of dicts with 'compound' and 'laps' keys
            base_lap_time: Base lap time in seconds
        """
        total_time = 0
        current_lap = 0
        stint_details = []
        
        for i, stint in enumerate(stints):
            stint_time = self.calculate_stint_time(
                compound=stint['compound'],
                start_lap=current_lap,
                stint_length=stint['laps'],
                base_lap_time=base_lap_time
            )
            
            stint_details.append({
                'stint': i + 1,
                'compound': stint['compound'],
                'start_lap': current_lap + 1,
                'end_lap': current_lap + stint['laps'],
                'stint_time': stint_time
            })
            
            total_time += stint_time
            current_lap += stint['laps']
            
            # Add pit stop time (except for last stint)
            if i < len(stints) - 1:
                total_time += self.pit_loss
        
        num_stops = len(stints) - 1
        
        return {
            'total_time': total_time,
            'total_laps': current_lap,
            'num_stops': num_stops,
            'total_pit_time': num_stops * self.pit_loss,
            'stint_details': stint_details
        }
    
    def compare_strategies(self,
                           strategies: Dict[str, List[Dict]],
                           base_lap_time: float) -> pd.DataFrame:
        """Compare multiple strategies."""
        results = []
        
        for name, stints in strategies.items():
            sim_result = self.simulate_strategy(stints, base_lap_time)
            
            results.append({
                'Strategy': name,
                'TotalTime': sim_result['total_time'],
                'NumStops': sim_result['num_stops'],
                'PitTimeLoss': sim_result['total_pit_time'],
                'Stints': ' -> '.join([f"{s['compound']}({s['end_lap']-s['start_lap']+1})" 
                                       for s in sim_result['stint_details']])
            })
        
        df = pd.DataFrame(results)
        df['GapToBest'] = df['TotalTime'] - df['TotalTime'].min()
        
        return df.sort_values('TotalTime')


# Initialize simulator
simulator = StrategySimulator(total_laps=57, pit_loss=21.0)

In [ ]:
# Compare different strategies for a 57-lap race
strategies = {
    '1-Stop (M-H)': [
        {'compound': 'MEDIUM', 'laps': 28},
        {'compound': 'HARD', 'laps': 29}
    ],
    '1-Stop (H-M)': [
        {'compound': 'HARD', 'laps': 30},
        {'compound': 'MEDIUM', 'laps': 27}
    ],
    '2-Stop (S-M-M)': [
        {'compound': 'SOFT', 'laps': 15},
        {'compound': 'MEDIUM', 'laps': 22},
        {'compound': 'MEDIUM', 'laps': 20}
    ],
    '2-Stop (M-M-S)': [
        {'compound': 'MEDIUM', 'laps': 20},
        {'compound': 'MEDIUM', 'laps': 22},
        {'compound': 'SOFT', 'laps': 15}
    ],
    '2-Stop (M-H-S)': [
        {'compound': 'MEDIUM', 'laps': 18},
        {'compound': 'HARD', 'laps': 25},
        {'compound': 'SOFT', 'laps': 14}
    ],
    '3-Stop (S-S-M-M)': [
        {'compound': 'SOFT', 'laps': 12},
        {'compound': 'SOFT', 'laps': 12},
        {'compound': 'MEDIUM', 'laps': 17},
        {'compound': 'MEDIUM', 'laps': 16}
    ]
}

comparison = simulator.compare_strategies(strategies, base_lap_time=92.0)
print("Strategy Comparison (57-lap race, 92s base lap time):")
comparison

In [ ]:
# Visualize strategy comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Total time comparison
colors = ['gold', 'silver', 'peru'] + ['steelblue'] * (len(comparison) - 3)
axes[0].barh(comparison['Strategy'], comparison['GapToBest'], color=colors[:len(comparison)])
axes[0].set_xlabel('Gap to Optimal Strategy (seconds)')
axes[0].set_title('Strategy Time Comparison')
axes[0].invert_yaxis()

# Pit time vs driving time
x = range(len(comparison))
width = 0.35

driving_time = comparison['TotalTime'] - comparison['PitTimeLoss']
axes[1].bar(x, driving_time, width, label='Driving Time', color='steelblue')
axes[1].bar(x, comparison['PitTimeLoss'], width, bottom=driving_time, label='Pit Time Loss', color='coral')

axes[1].set_xticks(x)
axes[1].set_xticklabels(comparison['Strategy'], rotation=45, ha='right')
axes[1].set_ylabel('Total Time (seconds)')
axes[1].set_title('Time Breakdown by Strategy')
axes[1].legend()

plt.tight_layout()
plt.show()

## 7. Safety Car Strategy Model

In [ ]:
class SafetyCarStrategyAdvisor:
    """Advise on pit strategy during safety car periods."""
    
    def __init__(self):
        # Typical pit stop time savings under SC (reduced pit lane delta)
        self.sc_pit_gain = 8.0  # seconds saved compared to green flag stop
        
    def should_pit_under_sc(self,
                            current_lap: int,
                            total_laps: int,
                            tire_age: int,
                            compound: str,
                            position: int,
                            planned_stops_remaining: int,
                            cars_ahead_pitting: int = 0) -> Dict:
        """Determine if driver should pit under safety car."""
        laps_remaining = total_laps - current_lap
        tire_limits = TIRE_LIFE_ESTIMATE.get(compound, TIRE_LIFE_ESTIMATE['MEDIUM'])
        
        factors = {
            'tire_life_remaining': tire_limits['max'] - tire_age,
            'can_finish_on_current': (tire_age + laps_remaining) <= tire_limits['max'],
            'fresh_tire_advantage': tire_age > tire_limits['optimal'],
            'track_position_cost': cars_ahead_pitting > 0,
            'planned_stop_remaining': planned_stops_remaining > 0
        }
        
        # Decision logic
        reasons = []
        score = 0
        
        # Must pit if can't finish on current tires
        if not factors['can_finish_on_current'] and planned_stops_remaining == 0:
            return {
                'recommendation': 'PIT - MANDATORY',
                'reason': 'Cannot finish race on current tires',
                'confidence': 0.95
            }
        
        # Strong pit if:
        # - Planned stop remaining AND tire age is high
        if planned_stops_remaining > 0 and factors['fresh_tire_advantage']:
            score += 3
            reasons.append('Planned stop due and tires degraded')
        
        # Moderate pit if:
        # - Free stop (cars ahead pitting) AND can benefit from fresh tires
        if cars_ahead_pitting > 0 and tire_age > 10:
            score += 2
            reasons.append('Free stop opportunity')
        
        # Slight pit if:
        # - Large tire advantage available
        if tire_age > tire_limits['optimal']:
            score += 1
            reasons.append('Significant fresh tire advantage')
        
        # Stay out if:
        # - Fresh tires AND losing track position
        if tire_age < 10 and cars_ahead_pitting == 0:
            score -= 2
            reasons.append('Fresh tires and would lose track position')
        
        # Late race considerations
        if laps_remaining < 15:
            if factors['can_finish_on_current']:
                score -= 1
                reasons.append('Late race, can finish on current tires')
        
        # Make recommendation
        if score >= 3:
            recommendation = 'PIT - STRONG'
            confidence = 0.85
        elif score >= 1:
            recommendation = 'PIT - CONSIDER'
            confidence = 0.65
        elif score <= -1:
            recommendation = 'STAY OUT - STRONG'
            confidence = 0.80
        else:
            recommendation = 'MARGINAL - TRACK POSITION'
            confidence = 0.50
        
        return {
            'recommendation': recommendation,
            'score': score,
            'reasons': reasons,
            'factors': factors,
            'confidence': confidence
        }
    
    def analyze_sc_scenarios(self, 
                             driver_state: Dict,
                             total_laps: int) -> pd.DataFrame:
        """Analyze what-if scenarios for SC deployment."""
        scenarios = []
        
        # Simulate SC at different race stages
        for sc_lap in range(10, total_laps - 5, 5):
            result = self.should_pit_under_sc(
                current_lap=sc_lap,
                total_laps=total_laps,
                tire_age=driver_state.get('tire_age', 15) + (sc_lap - driver_state.get('current_lap', 0)),
                compound=driver_state.get('compound', 'MEDIUM'),
                position=driver_state.get('position', 5),
                planned_stops_remaining=driver_state.get('stops_remaining', 1),
                cars_ahead_pitting=0
            )
            
            scenarios.append({
                'SC_Lap': sc_lap,
                'Recommendation': result['recommendation'],
                'Confidence': result['confidence'],
                'Score': result.get('score', 0)
            })
        
        return pd.DataFrame(scenarios)


# Initialize SC advisor
sc_advisor = SafetyCarStrategyAdvisor()

In [ ]:
# Test SC strategy advice
test_state = {
    'current_lap': 20,
    'tire_age': 18,
    'compound': 'MEDIUM',
    'position': 5,
    'stops_remaining': 1
}

sc_advice = sc_advisor.should_pit_under_sc(
    current_lap=20,
    total_laps=57,
    tire_age=18,
    compound='MEDIUM',
    position=5,
    planned_stops_remaining=1,
    cars_ahead_pitting=2
)

print("Safety Car Strategy Advice:")
print("=" * 50)
print(f"Recommendation: {sc_advice['recommendation']}")
print(f"Confidence: {sc_advice['confidence']:.0%}")
print(f"\nReasons:")
for reason in sc_advice.get('reasons', []):
    print(f"  - {reason}")

In [ ]:
# Analyze SC scenarios throughout the race
driver_state = {
    'current_lap': 1,
    'tire_age': 0,
    'compound': 'MEDIUM',
    'position': 5,
    'stops_remaining': 1
}

sc_scenarios = sc_advisor.analyze_sc_scenarios(driver_state, total_laps=57)
print("SC Deployment Scenario Analysis:")
sc_scenarios

In [ ]:
# Visualize SC strategy recommendations
fig, ax = plt.subplots(figsize=(12, 5))

colors = {
    'PIT - MANDATORY': 'red',
    'PIT - STRONG': 'orange',
    'PIT - CONSIDER': 'yellow',
    'MARGINAL - TRACK POSITION': 'lightgray',
    'STAY OUT - STRONG': 'green'
}

bar_colors = [colors.get(r, 'gray') for r in sc_scenarios['Recommendation']]
bars = ax.bar(sc_scenarios['SC_Lap'], sc_scenarios['Score'], color=bar_colors, edgecolor='black')

ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax.axhline(y=1, color='orange', linestyle='--', alpha=0.5, label='Consider pit threshold')
ax.axhline(y=3, color='red', linestyle='--', alpha=0.5, label='Strong pit threshold')

ax.set_xlabel('Safety Car Deployment Lap')
ax.set_ylabel('Strategy Score (positive = pit, negative = stay out)')
ax.set_title('Pit Strategy Recommendation by SC Deployment Lap')
ax.legend()

plt.tight_layout()
plt.show()

## 8. Integrated Strategy Optimizer

In [ ]:
class StrategyOptimizer:
    """Integrated pit stop strategy optimizer."""
    
    def __init__(self, 
                 total_laps: int,
                 pit_loss: float,
                 available_compounds: List[str] = None):
        self.total_laps = total_laps
        self.pit_loss = pit_loss
        self.compounds = available_compounds or ['SOFT', 'MEDIUM', 'HARD']
        self.simulator = StrategySimulator(total_laps, pit_loss)
        
    def generate_valid_strategies(self, min_stops: int = 1, max_stops: int = 3) -> List[List[Dict]]:
        """Generate all valid strategy combinations."""
        valid_strategies = []
        
        for num_stops in range(min_stops, max_stops + 1):
            num_stints = num_stops + 1
            
            # Generate compound combinations
            from itertools import product
            compound_combos = list(product(self.compounds, repeat=num_stints))
            
            # Validate: must use at least 2 different compounds
            valid_combos = [c for c in compound_combos if len(set(c)) >= 2]
            
            for combo in valid_combos:
                # Generate stint lengths that sum to total_laps
                stint_lengths = self._generate_stint_lengths(combo)
                
                for lengths in stint_lengths:
                    strategy = [
                        {'compound': c, 'laps': l} 
                        for c, l in zip(combo, lengths)
                    ]
                    valid_strategies.append(strategy)
        
        return valid_strategies
    
    def _generate_stint_lengths(self, compounds: Tuple[str], n_variants: int = 3) -> List[List[int]]:
        """Generate valid stint length combinations for given compounds."""
        num_stints = len(compounds)
        variants = []
        
        for _ in range(n_variants):
            lengths = []
            remaining_laps = self.total_laps
            
            for i, compound in enumerate(compounds):
                tire_limits = TIRE_LIFE_ESTIMATE.get(compound, TIRE_LIFE_ESTIMATE['MEDIUM'])
                
                if i == num_stints - 1:  # Last stint
                    stint_length = remaining_laps
                else:
                    # Random length within tire limits
                    max_stint = min(tire_limits['max'], remaining_laps - (num_stints - i - 1) * 10)
                    min_stint = max(tire_limits['min'], 10)
                    
                    if max_stint > min_stint:
                        stint_length = np.random.randint(min_stint, max_stint)
                    else:
                        stint_length = min_stint
                
                lengths.append(stint_length)
                remaining_laps -= stint_length
            
            if all(l > 0 for l in lengths):
                variants.append(lengths)
        
        return variants
    
    def find_optimal_strategy(self, 
                              base_lap_time: float,
                              deg_rates: Dict[str, float] = None,
                              n_strategies: int = 50) -> pd.DataFrame:
        """Find optimal strategy through simulation."""
        strategies = self.generate_valid_strategies()
        
        # Limit strategies for performance
        if len(strategies) > n_strategies:
            strategies = np.random.choice(strategies, n_strategies, replace=False).tolist()
        
        results = []
        
        for i, strategy in enumerate(strategies):
            sim_result = self.simulator.simulate_strategy(strategy, base_lap_time)
            
            # Create strategy name
            compounds_str = '-'.join([s['compound'][0] for s in strategy])
            lengths_str = '-'.join([str(s['laps']) for s in strategy])
            
            results.append({
                'Strategy': f"{compounds_str} ({lengths_str})",
                'NumStops': sim_result['num_stops'],
                'TotalTime': sim_result['total_time'],
                'Details': strategy
            })
        
        df = pd.DataFrame(results)
        df = df.sort_values('TotalTime').reset_index(drop=True)
        df['Rank'] = df.index + 1
        df['GapToBest'] = df['TotalTime'] - df['TotalTime'].iloc[0]
        
        return df
    
    def recommend_strategy(self,
                           base_lap_time: float,
                           risk_preference: str = 'balanced') -> Dict:
        """Recommend a strategy based on risk preference."""
        optimal_df = self.find_optimal_strategy(base_lap_time)
        
        if risk_preference == 'aggressive':
            # Prefer more stops for track position battles
            candidates = optimal_df[optimal_df['NumStops'] >= 2].head(3)
        elif risk_preference == 'conservative':
            # Prefer fewer stops
            candidates = optimal_df[optimal_df['NumStops'] <= 1].head(3)
        else:  # balanced
            candidates = optimal_df.head(3)
        
        if len(candidates) == 0:
            candidates = optimal_df.head(1)
        
        best = candidates.iloc[0]
        
        return {
            'recommended_strategy': best['Strategy'],
            'num_stops': best['NumStops'],
            'estimated_time': best['TotalTime'],
            'details': best['Details'],
            'alternatives': candidates.to_dict('records') if len(candidates) > 1 else []
        }


# Initialize optimizer
optimizer = StrategyOptimizer(
    total_laps=57,
    pit_loss=21.0,
    available_compounds=['SOFT', 'MEDIUM', 'HARD']
)

In [ ]:
# Find optimal strategies
optimal_strategies = optimizer.find_optimal_strategy(base_lap_time=92.0, n_strategies=100)

print("Top 10 Strategies:")
optimal_strategies[['Rank', 'Strategy', 'NumStops', 'TotalTime', 'GapToBest']].head(10)

In [ ]:
# Get recommendations for different risk profiles
print("\n" + "="*60)
for profile in ['conservative', 'balanced', 'aggressive']:
    rec = optimizer.recommend_strategy(base_lap_time=92.0, risk_preference=profile)
    print(f"\n{profile.upper()} Strategy:")
    print(f"  Recommended: {rec['recommended_strategy']}")
    print(f"  Stops: {rec['num_stops']}")
    print(f"  Est. Time: {rec['estimated_time']/60:.1f} minutes")

## 9. Save Models and Configurations

In [ ]:
import joblib
import json

# Create models directory
models_dir = Path('../saved_models/strategy')
models_dir.mkdir(parents=True, exist_ok=True)

# Save pit window model if trained
if pit_window_model.model is not None:
    joblib.dump(pit_window_model.model, models_dir / 'pit_window_model.joblib')
    joblib.dump(pit_window_model.compound_encoder, models_dir / 'compound_encoder.joblib')

# Save circuit pit loss data
with open(models_dir / 'circuit_pit_loss.json', 'w') as f:
    json.dump(CIRCUIT_PIT_LOSS, f, indent=2)

# Save tire life estimates
with open(models_dir / 'tire_life_estimates.json', 'w') as f:
    json.dump(TIRE_LIFE_ESTIMATE, f, indent=2)

print(f"Models and configs saved to {models_dir}")

## Summary

### Models and Components Developed:

1. **Pit Window Predictor**: ML model to predict optimal stint lengths
2. **Undercut Analyzer**: Detects and evaluates undercut/overcut opportunities
3. **Strategy Simulator**: Simulates race strategies with tire degradation
4. **Safety Car Advisor**: Real-time pit strategy advice during SC periods
5. **Strategy Optimizer**: Finds optimal strategies through simulation

### Key Capabilities:
- Predict pit windows based on tire compound and degradation
- Compare 1-stop, 2-stop, and 3-stop strategies
- React to safety car periods with optimal decisions
- Account for track position and tire age trade-offs

### Integration Points:
- Real-time lap time monitoring
- Position tracking for undercut analysis
- Weather/track condition updates
- Safety car detection from race control